In [1]:
import heapq as hq
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Test case 1
processes = ['P0', 'P1', 'P2','P3']
arrival_time = [0,0,0,0]
burst_time = [4,2,1,3]
waiting_time = [6,1,0,3]
turnaround_time = [10,3,1,6]
complete_time = [10,3,1,6]

In [3]:
# test case 2
processes = ['P0', 'P1', 'P2','P3','P4']
arrival_time = [0,1,2,3,7]
burst_time = [3,2,1,1,1]
waiting_time = [0,4,1,1,0]
turnaround_time = [3,6,2,2,1]
complete_time = [3,7,4,5,8]

In [4]:
# test case 3
processes = ['A', 'B', 'C','D','E']
arrival_time = [0,0,2,3,1]
burst_time = [3,2,1,4,3]
waiting_time = [3,0,0,6,5]
turnaround_time = [6,2,1,10,8]
complete_time = [6,2,3,13,9]

In [5]:
# test case 4
processes = ['P1','P2','P3','P4','P5','P6']
arrival_time = [0,25,30,60,100,105]
burst_time = [20,25,25,15,10,10]
waiting_time = [0,0,20,15,0,5]
turnaround_time = [20,25,45,30,10,15]
complete_time = [20,50,75,90,110,120]

In [6]:
# test case 5
processes = ['P1','P2','P3','P4','P5','P6']
arrival_time = [0,1,2,3,4,5]
burst_time = [8,4,2,1,3,2]
waiting_time = [12,5,0,1,6,0]
turnaround_time = [20,9,2,2,9,2]
complete_time = [20,10,4,5,13,7]

In [7]:
expected_answer = pd.DataFrame({'Process':processes, 'Arrival Time':arrival_time, 'Burst Time':burst_time, 
                               'Waiting Time':waiting_time, 'Turn Around Time':turnaround_time, 'Completion Time':complete_time})
expected_answer.sort_values(by=['Arrival Time', 'Burst Time'], ignore_index=True, inplace=True)

In [8]:
# generate a sorted timeline for the srtf algorithm

def sort_process_srtf(processes, arrival_time, burst_time):
    end_time = sum(burst_time)
    current_time = 0
    rem_time = burst_time.copy()
    processes_list = [ list(tup) for tup in list(zip(arrival_time, burst_time, processes, rem_time)) ]
    sorted_process_list = []
    hq.heapify(processes_list)
    
    while current_time < end_time or processes_list:
        available_processes_list = [process for process in processes_list if process[0] <= current_time]
        
        if available_processes_list:
            process = min(available_processes_list, key=lambda x: x[3])
            processes_list.remove(process)
            print("Process selected", process)
            
            process[3] = process[3] - 1
            
            if process[3] > 0:
                hq.heappush(processes_list, process)

            
            if len(sorted_process_list) == 0 or sorted_process_list[-1][2] != process[2]:
                # sorted_process_list.append( [ current_time, current_time + 1, process[2] ] ) # pocess start time, process end time, process name
                sorted_process_list.append( [ current_time, 1, process[2] ] )  # process start time, process duration, process name
            else:
                sorted_process_list[-1][1] += 1
            
            current_time +=1
            
        else:
            process = hq.nsmallest( 1, processes_list, key=lambda x: x[0] )[0]
            print("Process selected", process)
            
            process[3] = process[3] - 1
            
            if process[3] > 0:
                hq.heappush(processes_list, process)
            
            current_time = process[0]
            
            # sorted_process_list.append( [ current_time, current_time, process[2] ] )
            # sorted_process_list.append( [ current_time, current_time + 1, process[2] ] ) # pocess start time, process end time, process name
            
            sorted_process_list.append( [ current_time, 0, process[2] ] )  # process start time, process duration, process name
    
    print("\nSorted Processed List", sorted_process_list)
    return sorted_process_list

In [9]:
def generate_gantt_chart(timeline):
    end_at_prev = 0
    for i in timeline:
        start_at = i[0]
        end_at = i[1] + start_at
        
        if start_at != end_at_prev:
            print(f"{end_at_prev}-{start_at} : Idle | ", end="")
    
        print(f"{start_at}-{end_at} : Process {i[2]} | ", end="")
        
        end_at_prev = end_at

In [10]:
# writing a program for shortest job first scheduling algorithm with arrival times to return
# waiting time and turn around time for each process and average waiting time and average turn around time
# for all processes


def srtf(process, arrival_time, burst_time):    
    # for preemptive algorithms
    
    timeline = sort_process_srtf(processes, arrival_time, burst_time)
    processes_list = [ list(tup) for tup in list(zip( process, arrival_time, burst_time )) ]
    df = pd.DataFrame(processes_list, columns=['Process', 'Arrival Time', 'Burst Time' ])
    timeline_df = pd.DataFrame(timeline, columns=['Start Time', 'Duration', 'Process'])
    
    list_of_processes = df['Process'].tolist()
    for i in list_of_processes:
        temp = timeline_df.loc[timeline_df['Process'] == i, 'Start Time'] + timeline_df.loc[timeline_df['Process'] == i, 'Duration']
        df.loc[df['Process'] == i, 'Completion Time'] = temp.iloc[-1]

    df['Turn Around Time'] = df['Completion Time'] - df['Arrival Time']
    df['Waiting Time'] = df['Turn Around Time'] - df['Burst Time']    
    
    df = df[['Process', 'Arrival Time', 'Burst Time', 'Waiting Time', 'Turn Around Time', 'Completion Time']]
        
    print("\nProcess Details:")
    for i in range(len(df)):
        print(f"Process: {df['Process'].iloc[i]}, Wait Time: {df['Waiting Time'].iloc[i]}, Turn Around Time: {df['Turn Around Time'].iloc[i]}, Completion Time: {df['Completion Time'].iloc[i]}")
        
    print(f"\nAverage Waiting Time: {df['Waiting Time'].mean():.2f} ")
    print(f"Average Turn Around Time:  {df['Turn Around Time'].mean():.2f} ")
    return df.reset_index(drop=True) , timeline

In [11]:
actual_answer, timeline = srtf(processes, arrival_time, burst_time)

Process selected [0, 8, 'P1', 8]
Process selected [1, 4, 'P2', 4]
Process selected [2, 2, 'P3', 2]
Process selected [2, 2, 'P3', 1]
Process selected [3, 1, 'P4', 1]
Process selected [5, 2, 'P6', 2]
Process selected [5, 2, 'P6', 1]
Process selected [1, 4, 'P2', 3]
Process selected [1, 4, 'P2', 2]
Process selected [1, 4, 'P2', 1]
Process selected [4, 3, 'P5', 3]
Process selected [4, 3, 'P5', 2]
Process selected [4, 3, 'P5', 1]
Process selected [0, 8, 'P1', 7]
Process selected [0, 8, 'P1', 6]
Process selected [0, 8, 'P1', 5]
Process selected [0, 8, 'P1', 4]
Process selected [0, 8, 'P1', 3]
Process selected [0, 8, 'P1', 2]
Process selected [0, 8, 'P1', 1]

Sorted Processed List [[0, 1, 'P1'], [1, 1, 'P2'], [2, 2, 'P3'], [4, 1, 'P4'], [5, 2, 'P6'], [7, 3, 'P2'], [10, 3, 'P5'], [13, 7, 'P1']]

Process Details:
Process: P1, Wait Time: 12.0, Turn Around Time: 20.0, Completion Time: 20.0
Process: P2, Wait Time: 5.0, Turn Around Time: 9.0, Completion Time: 10.0
Process: P3, Wait Time: 0.0, Turn 

In [12]:
generate_gantt_chart(timeline)

0-1 : Process P1 | 1-2 : Process P2 | 2-4 : Process P3 | 4-5 : Process P4 | 5-7 : Process P6 | 7-10 : Process P2 | 10-13 : Process P5 | 13-20 : Process P1 | 

In [17]:
actual_answer

,Process,Arrival Time,Burst Time,Waiting Time,Turn Around Time,Completion Time
0,P1,0,8,12.0,20.0,20.0
1,P2,1,4,5.0,9.0,10.0
2,P3,2,2,0.0,2.0,4.0
3,P4,3,1,1.0,2.0,5.0
4,P5,4,3,6.0,9.0,13.0
5,P6,5,2,0.0,2.0,7.0


In [13]:
expected_answer

,Process,Arrival Time,Burst Time,Waiting Time,Turn Around Time,Completion Time
0,P1,0,8,12,20,20
1,P2,1,4,5,9,10
2,P3,2,2,0,2,4
3,P4,3,1,1,2,5
4,P5,4,3,6,9,13
5,P6,5,2,0,2,7


In [14]:
check_actual_answer = actual_answer.sort_values(by=['Arrival Time', 'Burst Time'], ignore_index=True)

In [15]:
diff = check_actual_answer.compare(expected_answer)

if diff.empty:
    print("Your answer is correct")
else:
    print("Your answer is incorrect")
    
diff

Your answer is correct


Empty DataFrame
Columns: []
Index: []

In [16]:
# # insert this line in srtf() function before uncommenting this cell and running it
# # df = df.astype({'Completion Time': 'int64', 'Turn Around Time': 'int64', 'Waiting Time': 'int64'})

# from pandas.testing import assert_frame_equal
# assert_frame_equal(check_actual_answer, expected_answer)